### Note
read_csv 에서 오류가 난다면 Analyze.py에

```pytho
engine = 'python'
```
를 추가해 줘보자.

```python
pd.read_csv('data/'+target_1 + '.csv',index_col='Date',engine='python')
```

또 csv 파일이 제대로 되어있는지 확인하여보자  
메모장으로 열었을 시에 제대로 ',' 로 구분되어야한다.

In [1]:
import SymbolList as sl
import Analyze as an
import numpy as np
import pandas as pd
import datetime as dt 
from statsmodels.formula.api import ols 
import seaborn as sns;  sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl

+ 한글 출력을 위한 폰트 설정, 각 PC마다 환경이 다름
+ 사용할 지표들의 이름

1. 인덱스들은 야후에서 잘 안받아짐, 수동으로 받은거 csv 로 쓸수 있게 다듬자.
2. 한글 폰트
3. 데이터 프레임 무결성 검증 루틴
4. 축소되지 않은 출력물
5. OLS 루틴

In [5]:
# matplot 설정
mpl.rcParams['axes.unicode_minus'] = False
# -*- coding: utf-8 -*-
plt.rc('font', family="LG PC")
# 리스트 설정
ten_kor = ['Samsung_Electronics','SK_hynix','LG_Electronics',
    '삼성SDI','엔씨소프트','카카오','삼성전기',
    'LG디스플레이','삼성SDS','네이버']
ten_US = [  'Apple','Microsoft','IBM','Oracle',
            'Amazon','Tesla','Alphabet','Facebook',
            'Cisco','타이완 반도체','Intel','오라클','SAP']
index_kor = '와이즈'
index_US = 'IXT'
#stan_US = ['IXT','VIX','BitCoin','MSCI','ten_Year_Treasury']
stan_US = ['XLK','VIX','BitCoin','MSCI','ten_Year_Treasury']

1. concat
2. dropna
3. number index
4. kor index를 다 -1 
5. process

+ 한국 기업 ~IXT+ VIX + 비트코인 + 미국 금리(DGS10) + MSCI

---

시간대가 문제이다. 
일단 

### OLS

In [20]:
import itertools as it
start='2016-01-04'
end='2018-12-28'
ols_ind=[]
for i in stan_US:
    tmp = pd.read_csv('data/'+i+'.csv',index_col='Date',engine='c')
    tmp = tmp[['Close']]
    tmp = tmp.loc[start:end]
    tmp.columns = [i]
    ols_ind.append(tmp)
    
ols_dep1 = pd.read_csv('data/Samsung_Electronics.csv',index_col='Date',engine='c')
ols_dep1 = ols_dep1[['Close']]
ols_dep1 = ols_dep1.loc[start:end]
ols_dep1.columns = ['Samsung_Electronics']

ols_dep2 = pd.read_csv('data/네이버.csv',index_col='Date',engine='c')
ols_dep2 = ols_dep2[['Close']]
ols_dep2 = ols_dep2.loc[start:end]
ols_dep2.columns = ['네이버']

### 1 Var OLS

In [43]:
for idx in range(len(ols_ind)):
    ols_df = pd.concat([ols_dep1, ols_ind[idx]], axis=1, sort=False)
    #ols_df = ols_df.pct_change()
    ols_df = ols_df[ols_df.select_dtypes(include=['number']).columns] * 100
    ols_df = ols_df.dropna()
    model_fit = ols('Samsung_Electronics'+'~1+'+stan_US[idx],data=ols_df).fit()
    
    print(model_fit.rsquared, end='')
    print(' :삼성전자 ~ '+ stan_US[idx])
    #print(model_fit.summary())
    
for idx in range(len(ols_ind)):
    ols_df = pd.concat([ols_dep2, ols_ind[idx]], axis=1, sort=False)
    #ols_df = ols_df.pct_change()
    ols_df = ols_df[ols_df.select_dtypes(include=['number']).columns] * 100
    ols_df = ols_df.dropna()
    model_fit = ols('네이버'+'~1+'+stan_US[idx],data=ols_df).fit()
    print(model_fit.rsquared, end='')
    print(' : 네이버 ~ ' + stan_US[idx])
    #print(model_fit.summary())

0.696594066543756 :삼성전자 ~ XLK
0.10984004203338982 :삼성전자 ~ VIX
0.5509360270498003 :삼성전자 ~ BitCoin
0.8723481270006285 :삼성전자 ~ MSCI
0.425379182528571 :삼성전자 ~ ten_Year_Treasury
0.010481082889032711 : 네이버 ~ XLK
0.29466118012643683 : 네이버 ~ VIX
0.04615024389757394 : 네이버 ~ BitCoin
0.20497433233128448 : 네이버 ~ MSCI
0.0062850960222518015 : 네이버 ~ ten_Year_Treasury


### 2 Var OLS

In [33]:
com = it.combinations(range(len(stan_US)), 2)
for idx in com:
    ols_df = pd.concat([ols_dep1, ols_ind[idx[0]],ols_ind[idx[1]]], axis=1, sort=False)
    #ols_df = ols_df.pct_change()
    ols_df = ols_df.dropna()
    model_fit = ols('Samsung_Electronics'+'~1+'+stan_US[idx[0]]+'+'+stan_US[idx[1]],data=ols_df).fit()
    print('%.5f'%(model_fit.rsquared), end='')
    print(' :삼성전자 ~ '+ stan_US[idx[0]] +' + '+stan_US[idx[1]])
    #print(model_fit.summary())
com = it.combinations(range(len(stan_US)), 2)  
for idx in com:
    ols_df = pd.concat([ols_dep2, ols_ind[idx[0]],ols_ind[idx[1]]], axis=1, sort=False)
    #ols_df = ols_df.pct_change()
    ols_df = ols_df.dropna()
    model_fit = ols('네이버'+'~1+'+stan_US[idx[0]]+'+'+stan_US[idx[1]],data=ols_df).fit()
    print('%.5f'%(model_fit.rsquared), end='')
    print(' :네이버 ~ '+ stan_US[idx[0]] +' + '+stan_US[idx[1]])
    #print(model_fit.summary())

0.76909 :삼성전자 ~ XLK + VIX
0.71104 :삼성전자 ~ XLK + BitCoin
0.88257 :삼성전자 ~ XLK + MSCI
0.70558 :삼성전자 ~ XLK + ten_Year_Treasury
0.63078 :삼성전자 ~ VIX + BitCoin
0.87886 :삼성전자 ~ VIX + MSCI
0.59696 :삼성전자 ~ VIX + ten_Year_Treasury
0.87454 :삼성전자 ~ BitCoin + MSCI
0.60996 :삼성전자 ~ BitCoin + ten_Year_Treasury
0.88705 :삼성전자 ~ MSCI + ten_Year_Treasury
0.29846 :네이버 ~ XLK + VIX
0.05715 :네이버 ~ XLK + BitCoin
0.45340 :네이버 ~ XLK + MSCI
0.10498 :네이버 ~ XLK + ten_Year_Treasury
0.32590 :네이버 ~ VIX + BitCoin
0.39528 :네이버 ~ VIX + MSCI
0.29483 :네이버 ~ VIX + ten_Year_Treasury
0.28534 :네이버 ~ BitCoin + MSCI
0.11628 :네이버 ~ BitCoin + ten_Year_Treasury
0.39222 :네이버 ~ MSCI + ten_Year_Treasury


### 3 Var OLS

In [52]:
com = it.combinations(range(len(stan_US)), 3)
for idx in com:
    ols_df = pd.concat([ols_dep1, ols_ind[idx[0]],ols_ind[idx[1]],ols_ind[idx[2]]], axis=1, sort=False)
    ols_df = ols_df.pct_change()
    ols_df = ols_df.dropna()
    model_fit = ols('Samsung_Electronics'+'~1+'+stan_US[idx[0]]+'+'+stan_US[idx[1]]+'+'+stan_US[idx[2]],data=ols_df).fit()
    print('%.5f'%(model_fit.rsquared), end='')
    print(' :삼성전자 ~ '+ stan_US[idx[0]] +' + '+stan_US[idx[1]]+' + '+stan_US[idx[2]])
    #print(model_fit.summary())
com = it.combinations(range(len(stan_US)), 3)  
for idx in com:
    ols_df = pd.concat([ols_dep2, ols_ind[idx[0]],ols_ind[idx[1]],ols_ind[idx[2]]], axis=1, sort=False)
    ols_df = ols_df.pct_change()
    ols_df = ols_df.dropna()
    model_fit = ols('네이버'+'~1+'+stan_US[idx[0]]+'+'+stan_US[idx[1]] + '+'+stan_US[idx[2]],data=ols_df).fit()
    print('%.5f'%(model_fit.rsquared), end='')
    print(' :네이버 ~ '+ stan_US[idx[0]] +' + '+stan_US[idx[1]]+ ' + ' + stan_US[idx[2]])
    #print(model_fit.summary())

0.00822 :삼성전자 ~ XLK + VIX + BitCoin
0.04025 :삼성전자 ~ XLK + VIX + MSCI
0.00797 :삼성전자 ~ XLK + VIX + ten_Year_Treasury
0.03959 :삼성전자 ~ XLK + BitCoin + MSCI
0.00360 :삼성전자 ~ XLK + BitCoin + ten_Year_Treasury
0.04151 :삼성전자 ~ XLK + MSCI + ten_Year_Treasury
0.02730 :삼성전자 ~ VIX + BitCoin + MSCI
0.00829 :삼성전자 ~ VIX + BitCoin + ten_Year_Treasury
0.02610 :삼성전자 ~ VIX + MSCI + ten_Year_Treasury
0.02645 :삼성전자 ~ BitCoin + MSCI + ten_Year_Treasury
0.00642 :네이버 ~ XLK + VIX + BitCoin
0.00986 :네이버 ~ XLK + VIX + MSCI
0.00720 :네이버 ~ XLK + VIX + ten_Year_Treasury
0.00793 :네이버 ~ XLK + BitCoin + MSCI
0.00209 :네이버 ~ XLK + BitCoin + ten_Year_Treasury
0.00876 :네이버 ~ XLK + MSCI + ten_Year_Treasury
0.00848 :네이버 ~ VIX + BitCoin + MSCI
0.00729 :네이버 ~ VIX + BitCoin + ten_Year_Treasury
0.00876 :네이버 ~ VIX + MSCI + ten_Year_Treasury
0.00778 :네이버 ~ BitCoin + MSCI + ten_Year_Treasury


### 4 Var OLS

In [50]:
com = it.combinations(range(len(stan_US)), 4)
for idx in com:
    ols_df = pd.concat([ols_dep1, ols_ind[idx[0]],ols_ind[idx[1]],ols_ind[idx[2]],ols_ind[idx[3]]], axis=1, sort=False)
    ols_df = ols_df.pct_change()
    ols_df = ols_df.dropna()
    model_fit = ols('Samsung_Electronics'+'~1+'+stan_US[idx[0]]+'+'+stan_US[idx[1]]+'+'+stan_US[idx[2]],data=ols_df).fit()
    print('%.5f'%(model_fit.rsquared), end='')
    print(' :삼성전자 ~ '+ stan_US[idx[0]] +' + '+stan_US[idx[1]]+' + '+stan_US[idx[1]]+' + '+stan_US[idx[2]])
    #print(model_fit.summary())
com = it.combinations(range(len(stan_US)), 4)  
for idx in com:
    ols_df = pd.concat([ols_dep2, ols_ind[idx[0]],ols_ind[idx[1]],ols_ind[idx[2]],ols_ind[idx[3]]], axis=1, sort=False)
    ols_df = ols_df.pct_change()
    ols_df = ols_df.dropna()
    model_fit = ols('네이버'+'~1+'+stan_US[idx[0]]+'+'+stan_US[idx[1]] + '+'+stan_US[idx[2]],data=ols_df).fit()
    print('%.5f'%(model_fit.rsquared), end='')
    print(' :네이버 ~ '+ stan_US[idx[0]] +' + '+stan_US[idx[1]]+ ' + ' + stan_US[idx[1]]+' + '+stan_US[idx[2]])
    #print(model_fit.summary())

0.00822 :삼성전자 ~ XLK + VIX + VIX + BitCoin
0.00822 :삼성전자 ~ XLK + VIX + VIX + BitCoin
0.04025 :삼성전자 ~ XLK + VIX + VIX + MSCI
0.03959 :삼성전자 ~ XLK + BitCoin + BitCoin + MSCI
0.02730 :삼성전자 ~ VIX + BitCoin + BitCoin + MSCI
0.00642 :네이버 ~ XLK + VIX + VIX + BitCoin
0.00642 :네이버 ~ XLK + VIX + VIX + BitCoin
0.00986 :네이버 ~ XLK + VIX + VIX + MSCI
0.00793 :네이버 ~ XLK + BitCoin + BitCoin + MSCI
0.00848 :네이버 ~ VIX + BitCoin + BitCoin + MSCI


### 데이터 유효성 검사

In [ ]:
an.isValid(stan_US)

In [ ]:
an.isValid(ten_kor)
#an.isValid(ten_US)

## Symbol List 초기화 & 불러오기
+ IXT 같은 index 는 잘 안불러진다.
+ 잘 받았는지 검증하는 루틴이 필요함. 

In [3]:
df = sl.init_symbol_DF()
sl.add_symbol(df,'삼성SDS','018260.KS',init=True)
df = sl.init_symbol_DF()

DUP : 삼성SDS
DUP : Apple
DUP : Microsoft
DUP : IBM
DUP : Oracle
DUP : Amazon
DUP : Tesla
DUP : Alphabet
DUP : Facebook
DUP : Cisco
DUP : 타이완 반도체
DUP : Intel
DUP : 오라클
DUP : SAP
DUP : Samsung_Electronics
DUP : SK_hynix
DUP : LG_Electronics
DUP : 삼성SDI
DUP : 엔씨소프트
DUP : 삼성SDS
DUP : 카카오
DUP : 삼성전기
DUP : LG디스플레이
DUP : 네이버
ADD : XLK
DUP : BitCoin
DUP : MSCI
DUP : ten_Year_Treasury
DUP : VIX


## 2. Symbol List 에서 데이터 받기

TODO::이전에 받은 날짜에 이어서 붙이는거 추가하자.

In [4]:
sl.download()

Note::Processing  삼성SDS
  Pass::Alread up-to-date.
Note::Processing  Apple
  Pass::Alread up-to-date.
Note::Processing  Microsoft
  Pass::Alread up-to-date.
Note::Processing  IBM
  Pass::Alread up-to-date.
Note::Processing  Oracle
  Pass::Alread up-to-date.
Note::Processing  Amazon
  Pass::Alread up-to-date.
Note::Processing  Tesla
  Pass::Alread up-to-date.
Note::Processing  Alphabet
  Pass::Alread up-to-date.
Note::Processing  Facebook
  Pass::Alread up-to-date.
Note::Processing  Cisco
  Pass::Alread up-to-date.
Note::Processing  타이완 반도체
  Pass::Alread up-to-date.
Note::Processing  Intel
  Pass::Alread up-to-date.
Note::Processing  오라클
  Pass::Alread up-to-date.
Note::Processing  SAP
  Pass::Alread up-to-date.
Note::Processing  Samsung_Electronics
  Pass::Alread up-to-date.
Note::Processing  SK_hynix
  Pass::Alread up-to-date.
Note::Processing  LG_Electronics
  Pass::Alread up-to-date.
Note::Processing  삼성SDI
  Pass::Alread up-to-date.
Note::Processing  엔씨소프트
  Pass::Alread up-to-dat

## 3. 통계

+ how to calculate coskew and cokurtosis
https://stackoverflow.com/questions/41890870/how-to-calculate-coskew-and-cokurtosis

생각을 좀 해봐야겠다.

$$ K(X,X,Y,Y)= {E[(X - E[X])^2(Y - E[Y]^2)] \over \sigma_X^2\sigma_Y^2}$$

### 출력이 스크롤 안에 안 들어가게 하기
Cell -> All output (또는 Current outputs) -> toggle scrolling

### Joint plot
https://seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
for idx in range(0,len(ten_kor)):
    an.joint(ten_kor[idx],index_US)

In [ ]:
for idx in range(0,len(ten_US)):
    an.joint(ten_US[idx],index_kor)